# Models

And now to try and predict.

In [1]:
# Load in the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

Now to set up the data again. As before, refer to notebooks 1 and 2 for more information.

In [2]:
# Set up the data
lines = pd.read_csv('../data/All-seasons.csv')

lines = lines[lines.Season != 'Season']

lines[['Season', 'Episode']] = lines[['Season', 'Episode']].astype('int64')

lines['is_cartman'] = 0

lines.loc[lines.Character == 'Cartman', 'is_cartman'] = 1

lines.head(3)

,Season,Episode,Character,Line,is_cartman
0,10,1,Stan,"You guys, you guys! Chef is going away. \n",0
1,10,1,Kyle,Going away? For how long?\n,0
2,10,1,Stan,Forever.\n,0


### Corpus

The corpus is established here, with steps to convert everything to lowercase and remove punctuation from the end of each word.

In [3]:
import re, string

corpus = lines.Line.tolist()

for line in range(len(corpus)):
    corpus[line] = re.sub('\\n', '', corpus[line].rstrip()).lower()
    corpus[line] = " ".join(word.strip(string.punctuation) for word in corpus[line].split())
    
corpus[7:10]

["what's the meaning of life why are we here",
 "i hope you're making the right choice",
 "i'm gonna miss him i'm gonna miss chef and i...and i don't know how to tell him"]

<b>Improving the corpus by removing contractions and lemmatizing words</b>

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import contractions

Here is a loop to expand all contractions:

In [5]:
for line in range(len(corpus)):
    corpus[line] = contractions.fix(corpus[line])
                                  
corpus[7:10]

['what is the meaning of life why are we here',
 'i hope you are making the right choice',
 'I am going to miss him I am going to miss chef and i...and i do not know how to tell him']

Now a function to lemmatize all verbs and nouns:

In [6]:
lem = WordNetLemmatizer()

def lemmatize_lines(line):
    word_list = word_tokenize(line)
    
    word_list = [lem.lemmatize(w, pos='v') for w in word_list]
    
    lem_line = ' '.join([lem.lemmatize(w) for w in word_list])
    
    return lem_line

And finally, a loop using the previous function to execute the lemmatization:

In [7]:
for line in range(len(corpus)):
    corpus[line] = lemmatize_lines(corpus[line])
    
corpus[7:9]

['what be the mean of life why be we here',
 'i hope you be make the right choice']

### Stop words

The list of stop words as determined in notebook 2 needs to instantiated.

In [8]:
sw = ['be', 'you', 'i', 'to', 'the', 'do', 'it',\
        'a', 'we', 'that', 'and', 'have', 'go', 'what',\
        'get', 'of', 'this', 'in', 'on', 'all', 'just',\
        'for', 'he', 'know', 'will', 'but', 'with', 'so',\
        'they', 'now', 'well', "'s", 'guy', 'u', 'come',\
        'like', 'there', 'at', 'would', 'who', 'him',\
        'them', 'his', 'thing', 'where', 'should', 'an',\
        'please', 'maybe', 'their', 'even', 'any', 'than']

### Word vectors and Data splits

The words and the corpus have been preprocessed, now it's time to continue the setup by establishing the word vector and then splitting up the train and test sets. From there, we can tune different models and try to determine the best predictor. We first start with CountVectorizer to establish a basic bag-of-words, combined with a Naive Bayes algorithm for training.<br>
<br>
<b>Establishing the vector</b>

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [10]:
vectorizer = CountVectorizer(stop_words=sw, ngram_range=(1,3))

X = vectorizer.fit_transform(corpus)
y = lines.is_cartman

state = 3

# How many features are there?
X.shape[1]

536629

<b>Splitting the data and training Multinomial Naive Bayes</b><br>
Now the features, X, and the target labels, y, need to be split into test and training sets. Then we can fit the data to a Multinomial Naive Bayes model and cross validate on the training data before checking against the test data.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=state)

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

mnb = MultinomialNB()

cv_scores = cross_val_score(mnb, X_train, y_train, cv=5)

In [13]:
print('Scores: ', cv_scores)
print('Average score: ', np.mean(cv_scores))

Scores:  [0.86255542 0.86174929 0.86213847 0.86242693 0.8612175 ]
Average score:  0.8620175217113915


We would certainly like to see better results, but it's not terrible for the first run.<br>
<br>
The immediate next steps would be to try differt n-gram ranges for the count vectorizer, and different values of *alpha* for MultinomialNB. Let's experiment with n-grams first.

<b>Results for different n-gram ranges: MultinomialNB</b>

| ngram_range | Number of features | Mean cv score        |
|-------------|--------------------|----------------------|
|  (1,1)      | 20,939             | 0.856                 |
| (1,2)       | 232,654            | 0.860                 |
| (1,3)       | 536,629            | 0.862                 |
| (1,5)       | 1,060,181          | 0.849                 |
| (2,2)       | 211,715            | 0.812                 |
| (3,3)       | 303,975            | 0.720                 |

Increasing the n-grams to (1,3) improves the accuracy, while also greatly increasing the features. Anything beyond that only adds features without any added benefit.<br>
<br>
Now we can experiment with values of *alpha* using `GridSearchCV`.

In [14]:
from sklearn.model_selection import GridSearchCV

params = {'alpha': [1.1, 1.15, 1.17, 1.2, 1.25, 1.3]}

nb_grid = GridSearchCV(mnb, params, cv=5)

nb_grid.fit(X_train, y_train)

print('Best alpha: ', nb_grid.best_estimator_)
print('Best score: ', nb_grid.best_score_)

Best alpha:  MultinomialNB(alpha=1.15, class_prior=None, fit_prior=True)
Best score:  0.8629849843797238


*Alpha* helps, but only slightly.<br>
<br>
Let's check the confusion matrix.

In [22]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, nb_grid.predict(X_train))

array([[42748,    25],
       [ 3365,  3477]], dtype=int64)

In [23]:
from sklearn.metrics import classification_report

print(classification_report(y_train, nb_grid.predict(X_train), np.unique(y)))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96     42773
           1       0.99      0.51      0.67      6842

    accuracy                           0.93     49615
   macro avg       0.96      0.75      0.82     49615
weighted avg       0.94      0.93      0.92     49615



The results aren't bad. The predictions were labelled correctly 93% of the time, and the model was very good at correctly predicting the non-Cartman class. Cartman lines were correctly predicted about half the time.

<b>Common phrases</b><br>
We can also use the CountVectorizer along with different n-gram ranges to find common phrases used by Cartman. By using different ranges we can examine phrases of differing lengths.<br>
<br>
First, we use the `is_cartman` series to find the indices for all of Cartman's lines. These indices will then be used to select which arrays to pull using the `X` variable that was established earlier by fitting the corpus to the vectorizer.

In [29]:
ind = list(y[y ==1].index)

Next, we enumerate over the arrays searching for any words that appear in that document. This won't give the actual word, but will give the indice for that word that can then be referenced in the vectorizer vocabulary to find the actual word.

In [30]:
vocab_values = []
    
for i in ind:
    vec = X[i].toarray().tolist()
    vec = vec[0]
    for i, el in enumerate(vec):
        if el > 0:
            vocab_values.append(i)

The `vocabulary_` attribute provides a dictionary of every word used in the vectorizer along with indice that corresponds with that word. These keys and values are then switched in the `new_vocab` dictionary so that words can be referenced by searching for the indice as the dictionary key. Then, the indices from the `vocab_values` list are used to create a simple list, not a set, of all the words or phrases used by Cartman.

In [31]:
vocab = vectorizer.vocabulary_

new_vocab = dict([(v, k) for k, v in vocab.items()])

vocab_words = []
for i in vocab_values:
    word = new_vocab[i]
    vocab_words.append(word)

Finally, a simple counter can be used on the `vocab_words` list to find the most common words and phrases.

In [49]:
from collections import Counter

cartman_phrases = Counter(vocab_words)
cartman_phrases.most_common(10)

[('oh my god', 72),
 ('can not believe', 15),
 ('not tell me', 12),
 ('no no no', 12),
 ('can not wait', 10),
 ('can not let', 10),
 ('let me see', 9),
 ('dude can not', 9),
 ('no can not', 9),
 ('why can not', 9)]

<b>Most common words and phrases</b><br>
<br>

| Single words\: (1,1)        | Two word phrases\: (2,2)| Three word phrases\: (3,3)|
|-----------------------------|-------------------------|---------------------------|
| not                         |  can not                |  oh my god                |
| my                          |  my got                 | can not believe           |
| can                         |  oh my                  | not tell me               |
| me                          |  not want               | no no no                  |
| oh                          |  not think              | can not wait              |
| your                        | south park              | can not let               |
| no                          | no not                  | let me see                |
| yeah                        | why not                 | dude can not              |
| here                        | tell me                 | no can not                |
| right                       | no no                   | why can not               |

The entries in the table above aren't as distinct as I had hoped. All the negative words kind of get in the way but, on the other hand, it shows how central the negations are to the tone of the dialogue. The sentiment of these phrases would be much different if 'no' and 'not' were removed from the corpus. Also, the expanded contractions and other preprocessing techniques kind of dilute the phrases.

### Tf-idf

To try and make further improvements, we can use a weighted tf-idf vectorizer instead of a basic CountVectorizer. Stop words aren't as big of a concern here due to the inherent adjustments, but since we already have a list we may as well use it. Also, because this is a new vectorizer we will have to create new train and test splits.

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vect = TfidfVectorizer(stop_words=sw, ngram_range=(1,1))

X = tf_vect.fit_transform(corpus)
y = lines.is_cartman

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=state)

Now to run the new splits with MultinomialNB and check the scores.

In [34]:
params = {'alpha': [1.0, 1.25, 1.35, 1.4]}

t_nb_grid = GridSearchCV(mnb, params, cv=5)

t_nb_grid.fit(X_train, y_train)

print('Best alpha: ', t_nb_grid.best_estimator_)
print('Best score: ', t_nb_grid.best_score_)

Best alpha:  MultinomialNB(alpha=1.35, class_prior=None, fit_prior=True)
Best score:  0.8624811045046861


In [35]:
confusion_matrix(y_train, t_nb_grid.predict(X_train))

array([[42766,     7],
       [ 6733,   109]], dtype=int64)

In [37]:
print(classification_report(y_train, t_nb_grid.predict(X_train), np.unique(y)))

              precision    recall  f1-score   support

           0       0.86      1.00      0.93     42773
           1       0.94      0.02      0.03      6842

    accuracy                           0.86     49615
   macro avg       0.90      0.51      0.48     49615
weighted avg       0.87      0.86      0.80     49615



Combined with MultinomialNB, the tf-idf vectorizer has about the same accuracy compared to CountVectorizer, but the confusion matrix shows a significant decrease in the predictive ability. However, there may be other algorithms better suited to the weighted vectors.

### Trying other models

<b>Random Forest</b><br>
The results so far have been decent, but maybe there are other algorithms that will perform better than Naive Bayes. First, let's try Random Forest, still with the tf-idf vectors. With `oob_score` (out of bag) set to 'True', we use the out-of-bag samples as a sort of validation set.

In [38]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100,\
                            max_features='sqrt',\
                            oob_score=True,\
                            random_state=state,\
                            n_jobs=-1)

rf.fit(X_train, y_train)
print(rf.oob_score_)

0.8628237428197117


In [40]:
confusion_matrix(y_train, rf.predict(X_train))

array([[42693,    80],
       [  936,  5906]], dtype=int64)

In [42]:
print(classification_report(y_train, rf.predict(X_train), np.unique(y)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     42773
           1       0.99      0.86      0.92      6842

    accuracy                           0.98     49615
   macro avg       0.98      0.93      0.95     49615
weighted avg       0.98      0.98      0.98     49615



The oob score is pretty in line with all the other results we've seen, but the results from the classification report far exceed anything else seen so far. The predictions for the Cartman class have a high precision and a fairly high recall as well for an F1 score of 0.92.

<b>SVM</b><br>
We can also try Support Vector Machines.

In [43]:
from sklearn.svm import LinearSVC

svm = LinearSVC(max_iter=5000, random_state=3)

params = {'C': [0.5, 0.6, 0.7, 0.8]}

sv_grid = GridSearchCV(svm, params, cv=5)

sv_grid.fit(X_train, y_train)

print('Best C: ', sv_grid.best_estimator_)
print('Best score: ', sv_grid.best_score_)

Best C:  LinearSVC(C=0.6, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=5000,
          multi_class='ovr', penalty='l2', random_state=3, tol=0.0001,
          verbose=0)
Best score:  0.867439282475058


In [44]:
confusion_matrix(y_train, sv_grid.predict(X_train))

array([[42425,   348],
       [ 5051,  1791]], dtype=int64)

The accuracy crept closer to 87%, but the results overall are far below Random Forest.

<b>Logistic Regression</b>

In [45]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=4000, multi_class='multinomial', random_state=3)

params = {'C': [1.0, 1.5, 1.75],
         'solver': ['newton-cg', 'sag']}

log_grid = GridSearchCV(log_reg, params, cv=5)

log_grid.fit(X_train, y_train)

print('Best C: ', log_grid.best_estimator_)
print('Best score: ', log_grid.best_score_)

Best C:  LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=4000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=3, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)
Best score:  0.8678222311800867


In [46]:
confusion_matrix(y_train, log_grid.predict(X_train))

array([[42396,   377],
       [ 5546,  1296]], dtype=int64)

Playing around wih different GridSearch parameters yields some better results, but it's very similar to SVM.

<b>Boosting</b><br>
<br>
Here's GradientBoosting:

In [47]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(max_depth=3, n_estimators=150, learning_rate=0.05, random_state=state)

cv_boost = cross_val_score(gb, X_train, y_train, cv=5)

In [48]:
print('Scores: ', cv_boost)
print('Average score: ', np.mean(cv_boost))

Scores:  [0.86648529 0.86517533 0.86375088 0.86585366 0.86655916]
Average score:  0.8655648645006405


In [50]:
gb.fit(X_train, y_train)

confusion_matrix(y_train, gb.predict(X_train))

array([[42739,    34],
       [ 6525,   317]], dtype=int64)

Still more or less the same.<br>
<br>
Now AdaBoost

In [51]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

dt_b = DecisionTreeClassifier(max_depth=1, random_state=state)

adb = AdaBoostClassifier(base_estimator=dt_b, n_estimators=200)

adb.fit(X_train, y_train)

cv_ada = cross_val_score(adb, X_train, y_train, cv=5)

In [52]:
print('Scores: ', cv_ada)
print('Average score: ', np.mean(cv_ada))

Scores:  [0.86537686 0.87011286 0.86818502 0.86958275 0.86565209]
Average score:  0.8677819156527221


In [53]:
confusion_matrix(y_train, adb.predict(X_train))

array([[42241,   532],
       [ 5709,  1133]], dtype=int64)

A slight improvement, but still hasn't exceeded Random Forest, and the computational cost is much more severe.